<a href="https://colab.research.google.com/github/pedrovinchi1/BlockChain-and-Mining/blob/main/Classificationv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle

In [3]:
!mv kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d puneet6060/intel-image-classification

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
 94% 325M/346M [00:01<00:00, 234MB/s]
100% 346M/346M [00:01<00:00, 201MB/s]


In [6]:
!unzip -q intel-image-classification.zip -d intel-image-classification

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
import os
import itertools

# === CONFIGURAÇÕES GLOBAIS ===
caminho_dataset = './intel-image-classification'
caminho_treino = os.path.join(caminho_dataset, 'seg_train/seg_train')
caminho_validacao = os.path.join(caminho_dataset, 'seg_test/seg_test')
forma_img = (224, 224, 3)
tam_lote = 32
num_epocas_finetuning = 10

# === FUNÇÕES AUXILIARES ===
# Função para carregar imagens de treino
def obter_imagens_treino(caminho_imagens_treino, forma_img, tam_lote):
    gerador_imagens = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    imagens_treino = gerador_imagens.flow_from_directory(
        caminho_imagens_treino,
        target_size=forma_img[:2],
        batch_size=tam_lote,
        class_mode='categorical',
        shuffle=True
    )
    return imagens_treino

# Função para carregar imagens de validação
def obter_imagens_validacao(caminho_imagens_validacao, forma_img, tam_lote):
    gerador_imagens = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    imagens_validacao = gerador_imagens.flow_from_directory(
        caminho_imagens_validacao,
        target_size=forma_img[:2],
        batch_size=tam_lote,
        class_mode='categorical',
        shuffle=False
    )
    return imagens_validacao

# Função para criar o modelo InceptionV3
def obter_modelo_inception(num_classes):
    modelo_base = keras.applications.InceptionV3(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )
    modelo_base.trainable = False
    # Adiciona camadas personalizadas
    inputs = keras.Input(shape=(224, 224, 3))
    x = modelo_base(inputs, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
    modelo = keras.Model(inputs, outputs)
    modelo.summary()
    return modelo

# Função para compilar o modelo
def compilar_modelo(modelo, learning_rate=0.0001):
    otimizador = keras.optimizers.Adam(learning_rate=learning_rate)
    modelo.compile(optimizer=otimizador, loss='categorical_crossentropy', metrics=['accuracy'])
    return modelo

# Função para treinar o modelo
def treinar_modelo(modelo, imagens_treino, imagens_validacao, num_epocas):
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath='melhores_pesos.keras',
        save_best_only=True,
        monitor='val_accuracy',
        mode='max'
    )
    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    historico = modelo.fit(
        imagens_treino,
        epochs=num_epocas,
        validation_data=imagens_validacao,
        callbacks=[checkpoint_callback, early_stop]
    )
    return historico

# Função para avaliar o modelo
def avaliar_modelo(modelo, imagens_validacao):

    # Previsões
    previsoes = modelo.predict(imagens_validacao)
    previsoes_classes = np.argmax(previsoes, axis=1)
    verdadeiras_classes = imagens_validacao.classes
    nomes_classes = list(imagens_validacao.class_indices.keys())

    # Matriz de confusão
    matriz_confusao = confusion_matrix(verdadeiras_classes, previsoes_classes)
    print("Matriz de Confusão:")
    print(matriz_confusao)

    # Relatório de classificação
    relatorio_classificacao = classification_report(verdadeiras_classes, previsoes_classes, target_names=nomes_classes)
    print("Relatório de Classificação:")
    print(relatorio_classificacao)

    # Acurácia
    acuracia = accuracy_score(verdadeiras_classes, previsoes_classes)
    print(f"Acurácia: {acuracia:.4f}")
    # Plotar matriz de confusão

    plot_matriz_confusao(matriz_confusao, nomes_classes)

# Função para plotar a matriz de confusão
def plot_matriz_confusao(matriz_confusao, nomes_classes):
    plt.figure(figsize=(8, 6))
    plt.imshow(matriz_confusao, interpolation='nearest', cmap='Blues')
    plt.title('Matriz de Confusão')
    plt.colorbar()
    tick_marks = np.arange(len(nomes_classes))
    plt.xticks(tick_marks, nomes_classes, rotation=45)
    plt.yticks(tick_marks, nomes_classes)
    fmt = 'd'
    thresh = matriz_confusao.max() / 2.
    for i, j in itertools.product(range(matriz_confusao.shape[0]), range(matriz_confusao.shape[1])):
        plt.text(j, i, format(matriz_confusao[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matriz_confusao[i, j] > thresh else "black")
    plt.ylabel('Verdadeiro')
    plt.xlabel('Previsto')
    plt.tight_layout()
    plt.show()

# Função para plotar o histórico de treinamento
def plot_historico(historico):
    plt.figure(figsize=(12, 5))

    # Gráfico de acurácia
    plt.subplot(1, 2, 1)
    plt.plot(historico.history['accuracy'], label='Treino')
    plt.plot(historico.history['val_accuracy'], label='Validação')
    plt.title('Acurácia')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()

    # Gráfico de perda
    plt.subplot(1, 2, 2)
    plt.plot(historico.history['loss'], label='Treino')
    plt.plot(historico.history['val_loss'], label='Validação')
    plt.title('Perda')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.show()

# Função para realizar o fine-tuning do modelo
def fine_tuning_modelo(modelo, imagens_treino, imagens_validacao):
    # Descongelar algumas camadas do modelo base
    for layer in modelo.layers[-20:]:
        layer.trainable = True
    # Recompilar o modelo com uma taxa de aprendizado menor
    otimizador_finetuning = keras.optimizers.Adam(learning_rate=1e-5)
    modelo.compile(optimizer=otimizador_finetuning, loss='categorical_crossentropy', metrics=['accuracy'])
    # Definir callbacks
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath='melhores_pesos_finetuning.keras',
        save_best_only=True,
        monitor='val_accuracy',
        mode='max'
    )
    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    # Continuar o treinamento
    historico_finetuning = modelo.fit(
        imagens_treino,
        epochs=num_epocas_finetuning,  # Menos épocas para fine-tuning
        validation_data=imagens_validacao,
        callbacks=[checkpoint_callback, early_stop]
    )
    return historico_finetuning

# === PIPELINE PRINCIPAL ===
# Carregamento de dados
imagens_treino = obter_imagens_treino(caminho_treino, forma_img, tam_lote)
imagens_validacao = obter_imagens_validacao(caminho_validacao, forma_img, tam_lote)

# Criação e compilação do modelo
modelo = obter_modelo_inception(imagens_treino.num_classes)
modelo = compilar_modelo(modelo)

# Fine-tuning do modelo
historico_finetuning = fine_tuning_modelo(modelo, imagens_treino, imagens_validacao)

# Plotar gráficos do fine-tuning
plot_historico(historico_finetuning)

# Avaliar o modelo após o fine-tuning
avaliar_modelo(modelo, imagens_validacao)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_v3 (Functional)            │ (None, 5, 5, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │          12,294 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,815,078 (83.22 MB)

 Trainable params: 12,294 (48.02 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

Epoch 1/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 7586s 17s/step - accuracy: 0.4810 - loss: 1.3007 - val_accuracy: 0.8913 - val_loss: 0.3412
Epoch 2/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 7498s 17s/step - accuracy: 0.8434 - loss: 0.4519 - val_accuracy: 0.9103 - val_loss: 0.2472
Epoch 3/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 7453s 17s/step - accuracy: 0.8702 - loss: 0.3689 - val_accuracy: 0.9177 - val_loss: 0.2225
Epoch 4/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 7479s 17s/step - accuracy: 0.8934 - loss: 0.2990 - val_accuracy: 0.9267 - val_loss: 0.2074
Epoch 5/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 7516s 17s/step - accuracy: 0.8953 - loss: 0.2890 - val_accuracy: 0.9283 - val_loss: 0.1990
Epoch 6/10
334/439 ━━━━━━━━━━━━━━━━━━━━ 28:30 16s/step - accuracy: 0.9058 - loss: 0.2582